<a href="https://colab.research.google.com/github/chorltonm/fa-cup-upsets/blob/main/notebooks/models/rating_models_unseen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Libaries
import os
import sys
import json
import pandas as pd
import numpy as np
import importlib
import pandas_gbq

from google.cloud import bigquery
from google.oauth2 import service_account
from google.colab import drive
from google.colab import userdata

# Install scikit learn
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, classification_report


In [ ]:
# Install Trueskill library
!pip install trueskill
from trueskill import Rating, rate_1vs1

In [ ]:
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Change default ouput directory
os.chdir('/content/drive/MyDrive/birkbeck_msc-project/output_files')

# Change default ouput directory
os.chdir('/content/drive/MyDrive/birkbeck_msc-project/python_files')

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print("Current working directory:", current_directory)

# List the contents of the directory
contents = os.listdir(current_directory)

# Print the contents
print("Contents of the directory:", contents)

# Import user defined python functions. Used importlib as having stability issues with simple import and not picking up the files

spec = importlib.util.spec_from_file_location("create_model_results", "/content/drive/MyDrive/birkbeck_msc-project/python_files/create_model_results.py")
create_model_results = importlib.util.module_from_spec(spec)
spec.loader.exec_module(create_model_results)

#from create_model_results import create_model_results_df


spec = importlib.util.spec_from_file_location("ratings_functions", "/content/drive/MyDrive/birkbeck_msc-project/python_files/ratings_functions.py")
ratings_functions = importlib.util.module_from_spec(spec)
spec.loader.exec_module(ratings_functions)

In [ ]:
# Authentication credentials and keys
# Google Service Account
# Load the JSON key from local Google Collab file
key = json.load(open('/content/drive/MyDrive/service_account.json', 'r'))

# Authenticate using the loaded key
credentials = service_account.Credentials.from_service_account_info(key)

# Set up the BigQuery client with the credentials to project
client = bigquery.Client(credentials=credentials, project='birkbeck-msc-project-422917')

In [ ]:
def load_predictions_and_ratings_to_bigquery(client, predictions_df, fold_ratings_df, ratings_model):

  # Load predictions to Big Query
  # Specify the target table
  load_dataset_name = 'analysis_layer'
  load_table_name = 'ratings_predictions_unseen'
  load_table_ref = f"{load_dataset_name}.{load_table_name}"

  # Delete predicitons method already inserted
  delete_query = f"DELETE FROM `{load_dataset_name}.{load_table_name}` WHERE ratings_model = '{ratings_model}'"
  delete_job = client.query(delete_query)
  delete_result = delete_job.result()
  predictions_total_rows_deleted = delete_result.num_dml_affected_rows
  print(f"Big Query target predictions table {load_dataset_name}.{load_table_name} rows deleted: {predictions_total_rows_deleted}")

  # Insert predictions for ratings model data to the existing table
  job_config = bigquery.LoadJobConfig(
          write_disposition="WRITE_APPEND")
  load_job = client.load_table_from_dataframe(
          predictions_df, load_table_ref, job_config=job_config)
  # Wait for the job to complete
  load_job.result()
  predictions_num_rows_inserted = load_job.output_rows
  print(f"{predictions_num_rows_inserted} rows appended to predictions table {load_table_ref} successfully.")

  # Load fold ratings to Big Query

  # Specify the target table
  load_dataset_name = 'analysis_layer'
  load_table_name = 'ratings_unseen'
  load_table_ref = f"{load_dataset_name}.{load_table_name}"

  # Delete ratings method already inserted
  delete_query = f"DELETE FROM `{load_dataset_name}.{load_table_name}` WHERE ratings_model = '{ratings_model}'"
  delete_job = client.query(delete_query)
  delete_result = delete_job.result()
  ratings_total_rows_deleted = delete_result.num_dml_affected_rows
  print(f"Big Query target ratings table {load_dataset_name}.{load_table_name} rows deleted: {ratings_total_rows_deleted}")


  # Insert ratings for ratings model data to the existing table
  job_config = bigquery.LoadJobConfig(
          write_disposition="WRITE_APPEND")
  load_job = client.load_table_from_dataframe(
          ratings_df, load_table_ref, job_config=job_config)

  load_job.result()  # Wait for the job to complete

  ratings_num_rows_inserted = load_job.output_rows
  print(f"{ratings_num_rows_inserted} rows appended to ratings table {load_table_ref} successfully.")

  return


In [ ]:
def predict_winner_upset (home_team_no, away_team_no, home_team_league_level, away_team_league_level, ranked_ratings):
    home_team_info = next((info for info in ranked_ratings if info[0] == home_team_no), None)
    away_team_info = next((info for info in ranked_ratings if info[0] == away_team_no), None)

    if home_team_info is None or away_team_info is None:
        raise ValueError("Team number not found in ratings list.")

    home_team_rating, home_team_rank = home_team_info[1], home_team_info[2]
    away_team_rating, away_team_rank = away_team_info[1], away_team_info[2]

    # Calculate the basic probability of the higer ranked team winning based proportionally as part of the total rank. Calculation reflects the fact lowest number is higher rank

    home_team_upset_probability = 1 - (home_team_rank / (home_team_rank + away_team_rank))
    away_team_upset_probability = 1 - (away_team_rank / (home_team_rank + away_team_rank))


    # Determine the predicted winner and if it's an upset
    if home_team_rating > away_team_rating:
        predicted_winner = home_team_no
        predicted_upset = 1 if home_team_league_level > away_team_league_level else 0
    else:
        predicted_winner = away_team_no
        predicted_upset = 1 if away_team_league_level > home_team_league_level else 0

    # Probaility of upset based on league level and rank differnce
    if home_team_league_level > away_team_league_level:
       upset_probability = home_team_upset_probability
    else:
       upset_probability = away_team_upset_probability

    return predicted_winner, predicted_upset, home_team_rating, away_team_rating, home_team_rank, away_team_rank, upset_probability

In [ ]:
def run_rating_models_single_split (train_data, test_data, ratings_function, ratings_model):
    # Initialize lists to store results
    train_accuracies = []
    train_recalls = []
    test_accuracies = []
    test_recalls = []
    all_y_train_true = []
    all_y_train_pred = []
    all_y_train_pred_proba = []
    all_y_test_true = []
    all_y_test_pred = []
    all_y_test_pred_proba = []
    ratings = []
    test_predictions = []

    # Call rating function
    if ratings_model in ['borda_count', 'average_rank', 'local_kemeny_optimisation']:
        model_ratings = ratings_function(1)  # Use 1 as a placeholder for fold and used when testign in unseen and single split
    else:
        model_ratings = ratings_function(train_data)

    # Add ranks based on the sorted order
    sorted_ratings = sorted(model_ratings, key=lambda x: x[1], reverse=True)
    ranked_ratings = [(team_no, rating, index + 1) for index, (team_no, rating) in enumerate(sorted_ratings)]

    # Append the ratings to the list
    ratings_list = [(ratings_model, 1, team_no, rating) for team_no, rating in model_ratings]
    ratings.extend(ratings_list)

    # Predict upsets for train data
    train_actual_upsets = []
    train_predicted_upsets = []
    train_upset_probabilities = []

    for _, row in train_data.iterrows():
        predicted_winner, predicted_upset, home_team_rating, away_team_rating, home_team_rank, away_team_rank, upset_probability = predict_winner_upset(
            row['home_team_no'], row['away_team_no'],
            row['home_team_league_level'], row['away_team_league_level'],
            ranked_ratings
        )
        actual_upset = row['actual_upset']
        train_actual_upsets.append(actual_upset)
        train_predicted_upsets.append(predicted_upset)
        train_upset_probabilities.append(upset_probability)

    # Calculate train metrics
    train_accuracy = accuracy_score(train_actual_upsets, train_predicted_upsets)
    train_recall = recall_score(train_actual_upsets, train_predicted_upsets)
    train_accuracies.append(train_accuracy)
    train_recalls.append(train_recall)
    all_y_train_true.extend(train_actual_upsets)
    all_y_train_pred.extend(train_predicted_upsets)
    all_y_train_pred_proba.extend(train_upset_probabilities)

    # Predict upsets for test data
    test_actual_upsets = []
    test_predicted_upsets = []
    test_upset_probabilities = []

    for _, row in test_data.iterrows():
        predicted_winner, predicted_upset, home_team_rating, away_team_rating, home_team_rank, away_team_rank, upset_probability = predict_winner_upset(
            row['home_team_no'], row['away_team_no'],
            row['home_team_league_level'], row['away_team_league_level'],
            ranked_ratings
        )
        actual_upset = row['actual_upset']
        test_actual_upsets.append(actual_upset)
        test_predicted_upsets.append(predicted_upset)
        test_upset_probabilities.append(upset_probability)

        test_predictions.append({
            'ratings_model': ratings_model,
            'match_id': row['match_id'],
            'home_team_no': row['home_team_no'],
            'home_team_league_level': row['home_team_league_level'],
            'away_team_no': row['away_team_no'],
            'away_team_league_level': row['away_team_league_level'],
            'home_team_rating': home_team_rating,
            'away_team_rating': away_team_rating,
            'home_team_rank': home_team_rank,
            'away_team_rank': away_team_rank,
            'predicted_winner': predicted_winner,
            'actual_winner': row['actual_winning_team_no'],
            'actual_upset': actual_upset,
            'predicted_upset': predicted_upset,
            'upset_probability': upset_probability,
        })

    # Calculate test metrics
    test_accuracy = accuracy_score(test_actual_upsets, test_predicted_upsets)
    test_recall = recall_score(test_actual_upsets, test_predicted_upsets)
    test_accuracies.append(test_accuracy)
    test_recalls.append(test_recall)
    all_y_test_true.extend(test_actual_upsets)
    all_y_test_pred.extend(test_predicted_upsets)
    all_y_test_pred_proba.extend(test_upset_probabilities)

    print(f"Train Accuracy: {train_accuracy:.3f}, Train Recall: {train_recall:.3f}")
    print(f"Test Accuracy: {test_accuracy:.3f}, Test Recall: {test_recall:.3f}")

    test_predictions_df = pd.DataFrame(test_predictions)

    # Create ratings DataFrame
    ratings_df = pd.DataFrame(ratings, columns=['ratings_model', 'fold_number', 'team_no', 'rating'])
    ratings_df = ratings_df.sort_values(['team_no', 'fold_number'])
    ratings_df['rank'] = ratings_df.groupby('fold_number')['rating'].rank(ascending=False, method='dense').astype(int)

    return ratings_df, test_predictions_df, all_y_train_true, all_y_train_pred, all_y_test_true, all_y_test_pred, train_accuracies, train_recalls, test_accuracies, test_recalls, all_y_train_pred_proba, all_y_test_pred_proba, ratings_model

In [ ]:
# Results & Rank Dataframes
all_ranks_df = pd.DataFrame(columns=['team_no','fold_number'])
display(all_ranks_df)

all_results_df = pd.DataFrame(columns=['metric_id', 'metric'])
display(all_results_df)

test_flag = 'Unseen'

In [ ]:
# Train and unseen data

fa_cup_scores_all = """
    SELECT * FROM preparation_layer.view_fa_cup_scores ORDER BY sort_order ASC
"""

fa_cup_scores_all_df = client.query(fa_cup_scores_all).to_dataframe()
display(fa_cup_scores_all_df)


fa_cup_scores__all_train_df = fa_cup_scores_all_df[~fa_cup_scores_all_df['season_year'].isin(['21/22', '22/23'])]
fa_cup_scores_test_all_unseen_df = fa_cup_scores_all_df[fa_cup_scores_all_df['season_year'].isin(['21/22', '22/23'])]

In [ ]:
# Basic Position Ratings
ratings_model = 'basic_position'
ratings_function = ratings_functions.basic_position_ratings
test_flag = 'Unseen'

# Call ratings function and train test split
(ratings_df, test_predictions_df, all_y_train_true, all_y_train_pred, all_y_test_true, all_y_test_pred,
 train_accuracies, train_recalls, test_accuracies, test_recalls,
 all_y_train_pred_proba, all_y_test_pred_proba, ratings_model) =  run_rating_models_single_split (fa_cup_scores__all_train_df, fa_cup_scores_test_all_unseen_df, ratings_function, ratings_model)


# Display results
test_predictions_df.index = range(1, len(test_predictions_df) + 1)
test_predictions_df_basic_position = test_predictions_df.index = range(1, len(test_predictions_df) + 1)
display(test_predictions_df_basic_position)

ratings_df.index = range(1, len(ratings_df) + 1)
display(ratings_df)

# Load predictions and ratinbgs to Google Big Query
load_predictions_and_ratings_to_bigquery(client, test_predictions_df, ratings_df, ratings_model)

# Create dataframe to compare rank and use for final rank aggregation
rank_df = ratings_df[['team_no', 'fold_number', 'rank']]
rank_df = rank_df.rename(columns={'rank': ratings_model})
all_ranks_df = all_ranks_df.merge(rank_df, on=['team_no','fold_number'], how='outer')
display(all_ranks_df)

# Create confusion matrix from results
model_name_ranking = ratings_model
results_df, cm_fig, roc_fig = create_model_results.create_model_results_df (all_y_train_true, all_y_train_pred, all_y_test_true, all_y_test_pred, train_accuracies, train_recalls, test_accuracies, test_recalls, all_y_train_pred_proba, all_y_test_pred_proba, model_name_ranking, test_flag)

results_df = results_df.reset_index()
results_df['metric_id'] = results_df.index + 1
results_df = results_df[['metric_id', 'metric', ratings_model]]

all_results_df = all_results_df.merge(results_df, on=['metric_id', 'metric'], how='outer')
display(all_results_df)



In [ ]:
# Massey Ratings
ratings_model = 'massey'
ratings_function = ratings_functions.massey_ratings

# Call ratings function and train test split
(ratings_df, test_predictions_df, all_y_train_true, all_y_train_pred, all_y_test_true, all_y_test_pred,
 train_accuracies, train_recalls, test_accuracies, test_recalls,
 all_y_train_pred_proba, all_y_test_pred_proba, ratings_model) =  run_rating_models_single_split (fa_cup_scores__all_train_df, fa_cup_scores_test_all_unseen_df, ratings_function, ratings_model)


# Display results
test_predictions_df.index = range(1, len(test_predictions_df) + 1)
test_predictions_df_massey = test_predictions_df.index = range(1, len(test_predictions_df) + 1)
display(test_predictions_df_massey)

ratings_df.index = range(1, len(ratings_df) + 1)
display(ratings_df)

# Load predictions and ratinbgs to Google Big Query
load_predictions_and_ratings_to_bigquery(client, test_predictions_df, ratings_df, ratings_model)

# Create dataframe to compare rank and use for final rank aggregation
rank_df = ratings_df[['team_no', 'fold_number', 'rank']]
rank_df = rank_df.rename(columns={'rank': ratings_model})
all_ranks_df = all_ranks_df.merge(rank_df, on=['team_no','fold_number'], how='outer')
display(all_ranks_df)

# Create confusion matrix from results
model_name_ranking = ratings_model
results_df, cm_fig, roc_fig = create_model_results.create_model_results_df (all_y_train_true, all_y_train_pred, all_y_test_true, all_y_test_pred, train_accuracies, train_recalls, test_accuracies, test_recalls, all_y_train_pred_proba, all_y_test_pred_proba, model_name_ranking, test_flag)

results_df = results_df.reset_index()
results_df['metric_id'] = results_df.index + 1
results_df = results_df[['metric_id', 'metric', ratings_model]]

all_results_df = all_results_df.merge(results_df, on=['metric_id', 'metric'], how='outer')
display(all_results_df)

In [ ]:
# Colley Ratings
ratings_model = 'colley'
ratings_function = ratings_functions.colley_ratings

# Call ratings function and train test split
(ratings_df, test_predictions_df, all_y_train_true, all_y_train_pred, all_y_test_true, all_y_test_pred,
 train_accuracies, train_recalls, test_accuracies, test_recalls,
 all_y_train_pred_proba, all_y_test_pred_proba, ratings_model) =  run_rating_models_single_split (fa_cup_scores__all_train_df, fa_cup_scores_test_all_unseen_df, ratings_function, ratings_model)

# Display results
test_predictions_df.index = range(1, len(test_predictions_df) + 1)
test_predictions_df_colley = test_predictions_df.index = range(1, len(test_predictions_df) + 1)
display(test_predictions_df_colley)

ratings_df.index = range(1, len(ratings_df) + 1)
display(ratings_df)

# Load predictions and ratinbgs to Google Big Query
load_predictions_and_ratings_to_bigquery(client, test_predictions_df, ratings_df, ratings_model)

# Create dataframe to compare rank and use for final rank aggregation
rank_df = ratings_df[['team_no', 'fold_number', 'rank']]
rank_df = rank_df.rename(columns={'rank': ratings_model})
all_ranks_df = all_ranks_df.merge(rank_df, on=['team_no','fold_number'], how='outer')
display(all_ranks_df)

# Create confusion matrix from results
model_name_ranking = ratings_model
results_df, cm_fig, roc_fig = create_model_results.create_model_results_df (all_y_train_true, all_y_train_pred, all_y_test_true, all_y_test_pred, train_accuracies, train_recalls, test_accuracies, test_recalls, all_y_train_pred_proba, all_y_test_pred_proba, model_name_ranking, test_flag)

results_df = results_df.reset_index()
results_df['metric_id'] = results_df.index + 1
results_df = results_df[['metric_id', 'metric', ratings_model]]

all_results_df = all_results_df.merge(results_df, on=['metric_id', 'metric'], how='outer')
display(all_results_df)

In [ ]:
# Keener Ratings
ratings_model = 'keener'
ratings_function = ratings_functions.keener_ratings

# Call ratings function and train test split
(ratings_df, test_predictions_df, all_y_train_true, all_y_train_pred, all_y_test_true, all_y_test_pred,
 train_accuracies, train_recalls, test_accuracies, test_recalls,
 all_y_train_pred_proba, all_y_test_pred_proba, ratings_model) =  run_rating_models_single_split (fa_cup_scores__all_train_df, fa_cup_scores_test_all_unseen_df, ratings_function, ratings_model)

# Display results
test_predictions_df.index = range(1, len(test_predictions_df) + 1)
test_predictions_df_keener = test_predictions_df.index = range(1, len(test_predictions_df) + 1)
display(test_predictions_df_keener)

ratings_df.index = range(1, len(ratings_df) + 1)
display(ratings_df)

# Load predictions and ratinbgs to Google Big Query
load_predictions_and_ratings_to_bigquery(client, test_predictions_df, ratings_df, ratings_model)

# Create dataframe to compare rank and use for final rank aggregation
rank_df = ratings_df[['team_no', 'fold_number', 'rank']]
rank_df = rank_df.rename(columns={'rank': ratings_model})
all_ranks_df = all_ranks_df.merge(rank_df, on=['team_no','fold_number'], how='outer')
display(all_ranks_df)

# Create confusion matrix from results
model_name_ranking = ratings_model
results_df, cm_fig, roc_fig = create_model_results.create_model_results_df (all_y_train_true, all_y_train_pred, all_y_test_true, all_y_test_pred, train_accuracies, train_recalls, test_accuracies, test_recalls, all_y_train_pred_proba, all_y_test_pred_proba, model_name_ranking, test_flag)

results_df = results_df.reset_index()
results_df['metric_id'] = results_df.index + 1
results_df = results_df[['metric_id', 'metric', ratings_model]]

all_results_df = all_results_df.merge(results_df, on=['metric_id', 'metric'], how='outer')
display(all_results_df)

In [ ]:
# Trueskill Ratings
ratings_model = 'trueskill'
ratings_function = ratings_functions.trueskill_ratings

# Call ratings function and train test split
(ratings_df, test_predictions_df, all_y_train_true, all_y_train_pred, all_y_test_true, all_y_test_pred,
 train_accuracies, train_recalls, test_accuracies, test_recalls,
 all_y_train_pred_proba, all_y_test_pred_proba, ratings_model) =  run_rating_models_single_split (fa_cup_scores__all_train_df, fa_cup_scores_test_all_unseen_df, ratings_function, ratings_model)

# Display results
test_predictions_df.index = range(1, len(test_predictions_df) + 1)
test_predictions_df_trueskill = test_predictions_df.index = range(1, len(test_predictions_df) + 1)
display(test_predictions_df_trueskill)

ratings_df.index = range(1, len(ratings_df) + 1)
display(ratings_df)

# Load predictions and ratinbgs to Google Big Query
load_predictions_and_ratings_to_bigquery(client, test_predictions_df, ratings_df, ratings_model)

# Create dataframe to compare rank and use for final rank aggregation
rank_df = ratings_df[['team_no', 'fold_number', 'rank']]
rank_df = rank_df.rename(columns={'rank': ratings_model})
all_ranks_df = all_ranks_df.merge(rank_df, on=['team_no','fold_number'], how='outer')
display(all_ranks_df)

# Create confusion matrix from results
model_name_ranking = ratings_model
results_df, cm_fig, roc_fig = create_model_results.create_model_results_df (all_y_train_true, all_y_train_pred, all_y_test_true, all_y_test_pred, train_accuracies, train_recalls, test_accuracies, test_recalls, all_y_train_pred_proba, all_y_test_pred_proba, model_name_ranking, test_flag)

results_df = results_df.reset_index()
results_df['metric_id'] = results_df.index + 1
results_df = results_df[['metric_id', 'metric', ratings_model]]

all_results_df = all_results_df.merge(results_df, on=['metric_id', 'metric'], how='outer')
display(all_results_df)


In [ ]:
# Load all ranks df to Big Query
load_dataset_name = 'analysis_layer'
load_table_name = 'ratings_model_ranks_unseen'
full_table_name = f"{load_dataset_name}.{load_table_name}"
pandas_gbq.to_gbq(all_ranks_df, full_table_name, project_id='birkbeck-msc-project-422917', if_exists='replace')

# Load confusion matrix df to Big Query
load_dataset_name = 'analysis_layer'
load_table_name = 'ratings_model_confusion_matrix_results_unseen'
full_table_name = f"{load_dataset_name}.{load_table_name}"
pandas_gbq.to_gbq(all_results_df, full_table_name, project_id='birkbeck-msc-project-422917', if_exists='replace')



In [ ]:
# Rank Aggregation functions called after ratings function have run and results inserted into Big Query

def borda_count_aggregation (fold_counter):

    from google.cloud import bigquery

    # Authenticate using the loaded key
    credentials = service_account.Credentials.from_service_account_info(key)
    # Set up the BigQuery client with the credentials to project
    client = bigquery.Client(credentials=credentials, project='birkbeck-msc-project-422917')

    fold_number = fold_counter
    all_ranks = """
              select * from analysis_layer.ratings_model_ranks_unseen
              """

    all_ranks_df = client.query(all_ranks).to_dataframe()
    all_ranks_df_fold = all_ranks_df[all_ranks_df['fold_number'] == fold_number].sort_values('team_no', ascending=True)
    all_ranks_df_fold['rank_total'] = all_ranks_df_fold[['basic_position','massey','colley','keener','trueskill']].sum(axis=1)
    all_ranks_df_fold['overall_rank'] = all_ranks_df_fold['rank_total'].rank(method='first', ascending=True).astype(int)
    all_ranks_df_fold['rating'] = 1 / all_ranks_df_fold['overall_rank']


    # Create a list of tuples in the format (team_no, rating) for function
    ratings = list(zip(all_ranks_df_fold['team_no'], all_ranks_df_fold['rating']))

    return ratings

def average_rank_aggregation (fold_counter):

    from google.cloud import bigquery

    # Authenticate using the loaded key
    credentials = service_account.Credentials.from_service_account_info(key)
    # Set up the BigQuery client with the credentials to project
    client = bigquery.Client(credentials=credentials, project='birkbeck-msc-project-422917')

    fold_number = fold_counter
    all_ranks = """
              select * from analysis_layer.ratings_model_ranks_unseen
              """

    all_ranks_df = client.query(all_ranks).to_dataframe()
    all_ranks_df_fold = all_ranks_df[all_ranks_df['fold_number'] == fold_number].sort_values('team_no', ascending=True)
    all_ranks_df_fold['rank_average'] = all_ranks_df_fold[['basic_position','massey','colley','keener','trueskill']].mean(axis=1)
    all_ranks_df_fold['overall_rank'] = all_ranks_df_fold['rank_average'].rank(method='first', ascending=True).astype(int)
    all_ranks_df_fold['rating'] = 1 / all_ranks_df_fold['overall_rank']
    display(all_ranks_df_fold)


    # Create a list of tuples in the format (team_no, rating) for function
    ratings = list(zip(all_ranks_df_fold['team_no'], all_ranks_df_fold['rating']))

    return ratings


In [ ]:
# Borda Count Aggregation
ratings_model = 'borda_count'
ratings_function = borda_count_aggregation

# Call ratings function and train test split
(ratings_df, test_predictions_df, all_y_train_true, all_y_train_pred, all_y_test_true, all_y_test_pred,
 train_accuracies, train_recalls, test_accuracies, test_recalls,
 all_y_train_pred_proba, all_y_test_pred_proba, ratings_model) =  run_rating_models_single_split (fa_cup_scores__all_train_df, fa_cup_scores_test_all_unseen_df, ratings_function, ratings_model)

# Display results
test_predictions_df.index = range(1, len(test_predictions_df) + 1)
test_predictions_df_borda_count = test_predictions_df.index = range(1, len(test_predictions_df) + 1)
display(test_predictions_df_borda_count)

ratings_df.index = range(1, len(ratings_df) + 1)
display(ratings_df)

# Load predictions and ratinbgs to Google Big Query
load_predictions_and_ratings_to_bigquery(client, test_predictions_df, ratings_df, ratings_model)

# Create dataframe to compare rank and use for final rank aggregation
rank_df = ratings_df[['team_no', 'fold_number', 'rank']]
rank_df = rank_df.rename(columns={'rank': ratings_model})
all_ranks_df = all_ranks_df.merge(rank_df, on=['team_no','fold_number'], how='outer')
display(all_ranks_df)

# Create confusion matrix from results
model_name_ranking = ratings_model
results_df, cm_fig, roc_fig = create_model_results.create_model_results_df (all_y_train_true, all_y_train_pred, all_y_test_true, all_y_test_pred, train_accuracies, train_recalls, test_accuracies, test_recalls, all_y_train_pred_proba, all_y_test_pred_proba, model_name_ranking, test_flag)

results_df = results_df.reset_index()
results_df['metric_id'] = results_df.index + 1
results_df = results_df[['metric_id', 'metric', ratings_model]]

all_results_df = all_results_df.merge(results_df, on=['metric_id', 'metric'], how='outer')
display(all_results_df)

In [ ]:
# Average Rank Aggregation
ratings_model = 'borda_count'
ratings_function = average_rank_aggregation

# Call ratings function and train test split
(ratings_df, test_predictions_df, all_y_train_true, all_y_train_pred, all_y_test_true, all_y_test_pred,
 train_accuracies, train_recalls, test_accuracies, test_recalls,
 all_y_train_pred_proba, all_y_test_pred_proba, ratings_model) =  run_rating_models_single_split (fa_cup_scores__all_train_df, fa_cup_scores_test_all_unseen_df, ratings_function, ratings_model)


# Display results
test_predictions_df.index = range(1, len(test_predictions_df) + 1)
test_predictions_df_average_rank = test_predictions_df.index = range(1, len(test_predictions_df) + 1)
display(test_predictions_df_average_rank)

ratings_df.index = range(1, len(ratings_df) + 1)
display(ratings_df)

# Load predictions and ratinbgs to Google Big Query
load_predictions_and_ratings_to_bigquery(client, test_predictions_df, ratings_df, ratings_model)

# Create dataframe to compare rank and use for final rank aggregation
rank_df = ratings_df[['team_no', 'fold_number', 'rank']]
rank_df = rank_df.rename(columns={'rank': ratings_model})
all_ranks_df = all_ranks_df.merge(rank_df, on=['team_no','fold_number'], how='outer')
display(all_ranks_df)

# Create confusion matrix from results
model_name_ranking = ratings_model
results_df, cm_fig, roc_fig = create_model_results.create_model_results_df (all_y_train_true, all_y_train_pred, all_y_test_true, all_y_test_pred, train_accuracies, train_recalls, test_accuracies, test_recalls, all_y_train_pred_proba, all_y_test_pred_proba, model_name_ranking, test_flag)

results_df = results_df.reset_index()
results_df['metric_id'] = results_df.index + 1
results_df = results_df[['metric_id', 'metric', ratings_model]]

all_results_df = all_results_df.merge(results_df, on=['metric_id', 'metric'], how='outer')
display(all_results_df)

In [ ]:
# Load updated all ranks df to Big Query so Borda Count can be subsequently queried in local kemeny optimisation
load_dataset_name = 'analysis_layer'
load_table_name = 'ratings_model_ranks_unseen'
full_table_name = f"{load_dataset_name}.{load_table_name}"
pandas_gbq.to_gbq(all_ranks_df, full_table_name, project_id='birkbeck-msc-project-422917', if_exists='replace')

In [ ]:
# Local Kemeny Optimisation

def local_kemeny_optimisation (fold_number, max_iterations=10):

    def kendall_tau_distance(ranking1, ranking2):
        distance = 0
        for i in range(len(ranking1)):
            for j in range(i + 1, len(ranking1)):
                if (ranking1[i] < ranking1[j]) != (ranking2[i] < ranking2[j]):
                    distance += 1
        return distance

    def total_kendall_tau_distance(candidate, rankings):
        return sum(kendall_tau_distance(candidate, ranking) for ranking in rankings.values())

    def local_kemeny_optimisation(rankings, initial_ranking, max_iterations):
        n_items = len(initial_ranking)

        current_ranking = initial_ranking.copy()
        current_distance = total_kendall_tau_distance(current_ranking, rankings)

        for iteration in range(max_iterations):
            improved = False
            for i in range(n_items - 1):
                new_ranking = current_ranking.copy()
                new_ranking[i], new_ranking[i+1] = new_ranking[i+1], new_ranking[i]
                new_distance = total_kendall_tau_distance(new_ranking, rankings)

                if new_distance < current_distance:
                    current_ranking = new_ranking
                    current_distance = new_distance
                    improved = True
                    print(f"Iteration {iteration + 1}: Improved distance to {current_distance}")
                    break

            if not improved:
                print(f"Stopped after {iteration + 1} iterations: No improvement")
                break

        if iteration == max_iterations - 1:
            print(f"Stopped after reaching maximum iterations ({max_iterations})")

        return current_ranking

    # Authenticate using the loaded key
    # credentials = service_account.Credentials.from_service_account_info(key)
    # Set up the BigQuery client with the credentials to project
    client = bigquery.Client(credentials=credentials, project='birkbeck-msc-project-422917')

    # Query to get the data
    all_ranks_query = f"""
        SELECT * FROM analysis_layer.ratings_model_ranks_unseen WHERE fold_number = {fold_number}
    """

    all_ranks_df = client.query(all_ranks_query).to_dataframe()

    # Create dictionary of rankings
    rankings = {
        'basic_position': all_ranks_df['basic_position'].tolist(),
        'massey': all_ranks_df['massey'].tolist(),
        'colley': all_ranks_df['colley'].tolist(),
        'keener': all_ranks_df['keener'].tolist(),
        'trueskill': all_ranks_df['trueskill'].tolist()
    }

    # Use Borda count as initial ranking
    borda_count = all_ranks_df['borda_count'].tolist()

    # Perform local Kemeny optimization
    optimized_ranking = local_kemeny_optimisation(rankings, borda_count, max_iterations)
    print("\nTotal Kendall tau distance (Borda Count):", total_kendall_tau_distance(borda_count, rankings))
    print("Total Kendall tau distance (Optimized):", total_kendall_tau_distance(optimized_ranking, rankings))

    # Show which teams changed positions
    team_nos = all_ranks_df['team_no'].tolist()

    # Create a mapping from team number to rank for both rankings
    borda_mapping = {team: rank for rank, team in enumerate(borda_count)}
    optimized_mapping = {team: rank for rank, team in enumerate(optimized_ranking)}

    changes = []
    for team in team_nos:
        borda_rank = borda_mapping.get(team, -1)
        optimized_rank = optimized_mapping.get(team, -1)
        if borda_rank != optimized_rank:
            changes.append((team, borda_rank, optimized_rank))

    print("\nTeams that changed positions (Team, Old Position, New Position):")
    for change in changes:
        print(f"Team {change[0]}: {change[1]} -> {change[2]}")

    # Create a DataFrame with the optimized ranking
    optimized_df = pd.DataFrame({
        'team_no': team_nos,
        'overall_rank': optimized_ranking
    })

    # Calculate the rating based on the optimized ranking
    optimized_df['rating'] = 1 / optimized_df['overall_rank']

    # Create a list of tuples in the format (team_no, rating) for function
    ratings = list(zip(optimized_df['team_no'], optimized_df['rating']))

    return ratings


In [ ]:
# Local Kemeny Optimisation
ratings_model = 'local_kemeny_optimisation'
ratings_function = local_kemeny_optimisation

# Call ratings function and train test split
(ratings_df, test_predictions_df, all_y_train_true, all_y_train_pred, all_y_test_true, all_y_test_pred,
 train_accuracies, train_recalls, test_accuracies, test_recalls,
 all_y_train_pred_proba, all_y_test_pred_proba, ratings_model) =  run_rating_models_single_split (fa_cup_scores__all_train_df, fa_cup_scores_test_all_unseen_df, ratings_function, ratings_model)


# Display results
test_predictions_df.index = range(1, len(test_predictions_df) + 1)
test_predictions_df_local_kemeny_optimisation = test_predictions_df.index = range(1, len(test_predictions_df) + 1)
display(test_predictions_df_local_kemeny_optimisation)

ratings_df.index = range(1, len(ratings_df) + 1)
display(ratings_df)

# Load predictions and ratinbgs to Google Big Query
load_predictions_and_ratings_to_bigquery(client, test_predictions_df, ratings_df, ratings_model)

# Create dataframe to compare rank and use for final rank aggregation
rank_df = ratings_df[['team_no', 'fold_number', 'rank']]
rank_df = rank_df.rename(columns={'rank': ratings_model})
all_ranks_df = all_ranks_df.merge(rank_df, on=['team_no','fold_number'], how='outer')
display(all_ranks_df)

# Create confusion matrix from results
model_name_ranking = ratings_model
results_df, cm_fig, roc_fig = create_model_results.create_model_results_df (all_y_train_true, all_y_train_pred, all_y_test_true, all_y_test_pred, train_accuracies, train_recalls, test_accuracies, test_recalls, all_y_train_pred_proba, all_y_test_pred_proba, model_name_ranking, test_flag)

results_df = results_df.reset_index()
results_df['metric_id'] = results_df.index + 1
results_df = results_df[['metric_id', 'metric', ratings_model]]

all_results_df = all_results_df.merge(results_df, on=['metric_id', 'metric'], how='outer')
display(all_results_df)

In [ ]:
# Load final ranks including Local Kemeny Optimisation
load_dataset_name = 'analysis_layer'
load_table_name = 'ratings_model_ranks_unseen'
full_table_name = f"{load_dataset_name}.{load_table_name}"
pandas_gbq.to_gbq(all_ranks_df, full_table_name, project_id='birkbeck-msc-project-422917', if_exists='replace')

In [ ]:
# Change default ouput directory
os.chdir('/content/drive/MyDrive/birkbeck_msc-project/output_files')
# Save ranks to excel
all_results_df.to_excel("all_rating_ranking_results_unseen.xlsx")

In [ ]:
# Load updated Fold results to Google Big Query

# Merge fold results from each model
all_data_act_pred_df = pd.concat([test_predictions_df_basic_position, test_predictions_df_massey, test_predictions_df_colley, test_predictions_df_keener, test_predictions_df_borda_count, test_predictions_df_average_rank, test_predictions_df_local_kemeny_optimisation])
all_data_act_pred_df = all_data_act_pred_df.rename(columns=lambda x: x.lower().replace(' ','(_)').replace('(', '').replace(')', ''))
display(all_data_act_pred_df)

# Write to Excel
all_data_act_pred_df.to_excel("all_rating_ranking_model_all_predictions_unseen.xlsx")

# Load fold results data from Excel to Google BigQuery
fold_results_from_excel = pd.read_excel("all_rating_ranking_model_all_predictions_unseen.xlsx")
load_dataset_name = 'analysis_layer'
load_table_name = 'all_rating_ranking_model_all_predictions_unseen'
full_table_name = f"{load_dataset_name}.{load_table_name}"

pandas_gbq.to_gbq(fold_results_from_excel, full_table_name,
                  project_id='birkbeck-msc-project-422917',
                  if_exists='replace')

print(f"\nData loaded to BigQuery table: {full_table_name}")